In [1]:
import numpy as np
from scipy.special import jv
from scipy.integrate import quad
from sympy.physics.wigner import clebsch_gordan
from scipy.optimize import brentq
from scipy.interpolate import interp1d

# ---------- Utilities ----------

def first_bessel_zero(nu, s=1, x_min=0.1, x_max=50.0):
    def f(x):
        return jv(nu, x)

    zeros = []
    x_left = x_min
    step = 0.1
    while x_left < x_max and len(zeros) < s:
        x_right = x_left + step
        if f(x_left) * f(x_right) < 0:
            zeros.append(brentq(f, x_left, x_right))
        x_left = x_right

    if len(zeros) < s:
        raise RuntimeError(f"Less than {s} zeros found for J_{nu}(x)")

    return zeros[s - 1]

def normalization_constant(nu, zero):
    return 0.5 * (jv(nu + 1, zero)) ** 2

def radial_wavefunction(beta, nu, zero, norm_const):
    return beta**(-1.5) * jv(nu, zero * beta) / np.sqrt(norm_const)

def radial_integral(L, zeros, s=1):
    nu_L = np.sqrt(L * (L + 1) / 3 + 9 / 4)
    zero_L = zeros[(s, L)]
    C_L = normalization_constant(nu_L, zero_L)

    L2 = L - 2
    nu_L2 = np.sqrt(L2 * (L2 + 1) / 3 + 9 / 4)
    zero_L2 = zeros[(s, L2)]
    C_L2 = normalization_constant(nu_L2, zero_L2)

    def integrand(beta):
        psi_L = radial_wavefunction(beta, nu_L, zero_L, C_L)
        psi_L2 = radial_wavefunction(beta, nu_L2, zero_L2, C_L2)
        return psi_L * psi_L2 * beta**5

    result, _ = quad(integrand, 0, 1, limit=200, epsabs=1e-10)
    return result**2

def cg_squared(L):
    return float(clebsch_gordan(L - 2, 2, L, 0, 0, 0).evalf()) ** 2

def alaga_prefactor(L):
    return (L - 1) * (L + 2) / (2 * L + 1)

# ---------- Energy Levels ----------

def energy_levels(max_s, L_max):
    levels = {}
    zeros = {}
    for s in range(1, max_s + 1):
        for L in range(0, L_max + 2, 2):
            nu = np.sqrt(L * (L + 1) / 3 + 9 / 4)
            zero = first_bessel_zero(nu, s)
            levels[(s, L)] = zero**2
            zeros[(s, L)] = zero

    E0 = levels[(1, 0)]
    E2 = levels[(1, 2)]
    norm_levels = {(s, L): (levels[(s, L)] - E0) / (E2 - E0) for (s, L) in levels}

    return norm_levels, zeros

# ---------- B(E2) with Scaling ----------

def compute_b2_scaled(zeros, L_max, s=1, ref_points=None, normalize_first_transition=None):
    raw_be2 = {}
    for L in range(2, L_max + 2, 2):
        I2 = radial_integral(L, zeros, s)
        CG2 = cg_squared(L)
        A = alaga_prefactor(L)
        raw_be2[(L, L - 2)] = I2 * CG2 * A

    if ref_points:
        calc_vals = [raw_be2[k] for k in ref_points]
        target_vals = [ref_points[k] for k in ref_points]
        interp_func = interp1d(calc_vals, target_vals, fill_value='extrapolate', kind='linear')
        scaled_be2 = {k: float(interp_func(v)) for k, v in raw_be2.items()}
    elif normalize_first_transition:
        target_val = normalize_first_transition
        first_key = (2, 0)
        scale_factor = target_val / raw_be2[first_key]
        scaled_be2 = {k: v * scale_factor for k, v in raw_be2.items()}
    else:
        scaled_be2 = raw_be2

    return scaled_be2

# ---------- Main Program ----------

if __name__ == "__main__":
    try:
        L_max = int(input("Enter highest even L (e.g., 10): "))
        max_s = int(input("Enter number of bands s (e.g., 5): "))

        if L_max % 2 != 0 or L_max < 2:
            raise ValueError("L must be even and ≥ 2.")

        energies, zeros = energy_levels(max_s, L_max)

        print("\nX(5) Energy Levels (E(0⁺)=0, E(2⁺)=1):")
        for s in range(1, max_s + 1):
            print(f"\ns = {s} band:")
            for L in range(0, L_max + 2, 2):
                print(f"E(s={s}, L={L}⁺) = {energies[(s, L)]:.4f}")

        ref_s1 = {
            (2, 0): 100,
            (4, 2): 160,
            (6, 4): 198,
            (8, 6): 228,
            (10, 8): 251
        }
        print("\nB(E2) values for s=1 band with scaling:")
        be2_s1 = compute_b2_scaled(zeros, L_max, s=1, ref_points=ref_s1)
        for (L, L2) in sorted(be2_s1):
            print(f"B(E2; s=1 {L}⁺ → s=1 {L2}⁺) = {be2_s1[(L, L2)]:.2f}")

        if max_s >= 2:
            ref_s2 = {
                (2, 0): 80,
                (4, 2): 120,
                (6, 4): 147,
                (8, 6): 169,
                (10, 8): 189
            }
            print("\nB(E2) values for s=2 band with scaling:")
            be2_s2 = compute_b2_scaled(zeros, L_max, s=2, ref_points=ref_s2)
            for (L, L2) in sorted(be2_s2):
                print(f"B(E2; s=2 {L}⁺ → s=2 {L2}⁺) = {be2_s2[(L, L2)]:.2f}")

        if max_s >= 3:
            ref_s3 = {
                (2, 0): 73,
                (4, 2): 104,
                (6, 4): 125,
                (8, 6): 143
            }
            print("\nB(E2) values for s=3 band with scaling:")
            be2_s3 = compute_b2_scaled(zeros, L_max, s=3, ref_points=ref_s3)
            for (L, L2) in sorted(be2_s3):
                print(f"B(E2; s=3 {L}⁺ → s=3 {L2}⁺) = {be2_s3[(L, L2)]:.2f}")

        if max_s >= 4:
            ref_s4 = {
                (2, 0): 69,
                (4, 2): 96
            }
            print("\nB(E2) values for s=4 band with scaling:")
            be2_s4 = compute_b2_scaled(zeros, L_max, s=4, ref_points=ref_s4)
            for (L, L2) in sorted(be2_s4):
                print(f"B(E2; s=4 {L}⁺ → s=4 {L2}⁺) = {be2_s4[(L, L2)]:.2f}")

        for s in range(5, max_s + 1):
            print(f"\nB(E2) values for s={s} band with automatic scaling:")
            target_first = 65
            be2_auto = compute_b2_scaled(zeros, L_max, s=s, normalize_first_transition=target_first)
            for (L, L2) in sorted(be2_auto):
                print(f"B(E2; s={s} {L}⁺ → s={s} {L2}⁺) = {be2_auto[(L, L2)]:.2f}")

    except Exception as e:
        print("Error:", e)



X(5) Energy Levels (E(0⁺)=0, E(2⁺)=1):

s = 1 band:
E(s=1, L=0⁺) = 0.0000
E(s=1, L=2⁺) = 1.0000
E(s=1, L=4⁺) = 2.9035
E(s=1, L=6⁺) = 5.4295
E(s=1, L=8⁺) = 8.4834
E(s=1, L=10⁺) = 12.0272

s = 2 band:
E(s=2, L=0⁺) = 5.6485
E(s=2, L=2⁺) = 7.4500
E(s=2, L=4⁺) = 10.6892
E(s=2, L=6⁺) = 14.7511
E(s=2, L=8⁺) = 19.4413
E(s=2, L=10⁺) = 24.6871

s = 3 band:
E(s=3, L=0⁺) = 14.1195
E(s=3, L=2⁺) = 16.7164
E(s=3, L=4⁺) = 21.2712
E(s=3, L=6⁺) = 26.8317
E(s=3, L=8⁺) = 33.1030
E(s=3, L=10⁺) = 39.9793

s = 4 band:
E(s=4, L=0⁺) = 25.4137
E(s=4, L=2⁺) = 28.8045
E(s=4, L=4⁺) = 34.6695
E(s=4, L=6⁺) = 41.7170
E(s=4, L=8⁺) = 49.5511
E(s=4, L=10⁺) = 58.0325

s = 5 band:
E(s=5, L=0⁺) = 39.5314
E(s=5, L=2⁺) = 43.7156
E(s=5, L=4⁺) = 50.8884
E(s=5, L=6⁺) = 59.4180
E(s=5, L=8⁺) = 68.8067
E(s=5, L=10⁺) = 78.8813

s = 6 band:
E(s=6, L=0⁺) = 56.4725
E(s=6, L=2⁺) = 61.4498
E(s=6, L=4⁺) = 69.9296
E(s=6, L=6⁺) = 79.9388
E(s=6, L=8⁺) = 90.8776
E(s=6, L=10⁺) = 102.5392

B(E2) values for s=1 band with scaling:
B(E2; s=1 2⁺ 